# **ETL del archivo Lesiones_Victimas**

## **1.Cargando Las Librerías Necesarias**

In [74]:
import pandas as pd #Cargaremos pandas para manejar los datos de nuestros datasets,
import numpy as np #Cargaremos numpy para manejar los datos,
import gzip #Cargaremos gzip para poder abrir archivos con compresion gzip",
import datetime #Cargaremos datetime para manejar fechas",
from google.colab import drive #Cargaremos drive para poder acceder a los archivos de drive"
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
url01="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/lesiones.xlsx"
url02="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/lesiones_victimas.csv"

## **2.Fase ETL**

### 2.1.Carga de Datos

In [76]:
df_lesiones_victimas=pd.read_excel(url01,sheet_name="VICTIMAS")

In [77]:
df_lesiones_victimas.to_csv(url02,index=True,header=True,encoding='utf-8')

In [78]:
df_lesiones_victimas=pd.read_csv(url02,encoding='utf-8')

In [79]:
df_lesiones_victimas.head()

,Unnamed: 0,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD
3,3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD
4,4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD


### 2.2.Analisis Descriptivo De Los Datos

#### 2.2.1.Funciones para el analisis descriptivo de los datos

In [80]:
def check_df(dataset, head = 5):

  """
  Funcion creada para tener una vista general de la base de datos
  """

  print('*'*30 + 'Forma de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.shape, end = '\n'*2)#Tamaño de la base de datos

  print('*'*30 + 'Informacion general de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.info(), end = '\n'*2)# Informacion general

  print('*'*30 + 'NaN cantidad de nulos' + '*'*30, end = '\n'*2)
  print(dataset.isnull().sum(), end = '\n'*2)#Cuenta la cantidad de NaN por columna

#### 2.2.2.Descripcion De Los Datos

In [81]:
check_df(df_lesiones_victimas)

******************************Forma de la base de datos******************************

(27605, 10)

******************************Informacion general de la base de datos******************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        27605 non-null  int64 
 1   ID hecho          27605 non-null  object
 2   AAA               27605 non-null  int64 
 3   MM                27605 non-null  int64 
 4   DD                27605 non-null  int64 
 5   FECHA             27605 non-null  object
 6   VEHICULO_VICTIMA  27605 non-null  object
 7   SEXO              27605 non-null  object
 8   EDAD_VICTIMA      27605 non-null  object
 9   GRAVEDAD          27605 non-null  object
dtypes: int64(4), object(6)
memory usage: 2.1+ MB
None

******************************NaN cantidad de nulos******************************


### 2.3.Transformacion De Los Datos

#### 2.3.1.Funciones para la transformacion de los datos

In [82]:
def Transform_Hora(text):

  '''

  La funcion de Transform_Hora recibe como parametro un dato de cualquier tipo
  el cual pasara a trasnformarse en texto y devuelve un objeto de tipo datetime
  en formato de 24horas.
  Observando 3 formatos de datos en total 2 de formato fecha y un tipo de dato
  string entonces procedemos de la siguiente manera :

  format_12h= "%I:%M:%S %p"
  format_24h='%H:%M:%S'
  format_sd='sd'
  Para este ejercicico lo pasaremos a formato 24 horas que es el formato
   por defecto

  '''
  text=str(text)
  format_12h= "%I:%M:%S %p"
  format_24h='%H:%M:%S'
  if (text.count("p.m.") or text.count("a.m."))>0: #format 12h
    text=text.replace("p.m.","PM").replace("a.m.","AM")
    time12h = datetime.datetime.strptime(text,format_12h).time()
    time24h = time12h.strftime(format_24h)
    time24h=datetime.datetime.strptime(time24h,format_24h).time()
    return  time24h
  elif text.count(":")==2: #format 24h
    time24h = datetime.datetime.strptime(text, format_24h).time()
    return  time24h

  else:
    return float(np.nan)


def reemplazar_nan(value):
  '''

  La funcion reemplazar_nan busca reemplaza las palabras como 'SD' , 'sd o
  No Especificada' por
  valores NaN

  '''
  if value in ('SD', 'sd', 'No Especificada'):
      return float(np.nan)
  else:
      return value


def reemplazar_participacion(value):
  '''

  La funcion reemplazar_participacion busca reemplaza en las columnas designadas
  para los participanes del siniestro como ['MOTO','AUTO','TRANSPORTE_PUBLICO',
  'CAMION','CICLISTA']
  valores NaN

  '''
  if value=='x':
    return round(1,0)
  elif value=='0':
    return round(0,0)
  else:
      return value


def Transform_Dir(text):
  """

  Esta funcion esta diseñada para reordenar la columna LUGAR_DEL_HECHO teniendo
  como principla estructura teniendo como prefijo AV. y DR. y eliminando la coma
  normaliando las abreviaciones en general

  """
  if isinstance(text, str):
    text=str(text).upper()
    if "," in text:
      parts = [part.strip() for part in text.split(",")]
      if len(parts) == 2:
          text= f"{parts[1]} {parts[0]} "
      if len(parts) == 3:
          text= f"{parts[2]} {parts[1]} {parts[0]} "
    if "DR." in text:
      parts = [part.strip() for part in text.split("DR.")]

      if len(parts) == 2:
        text= f"DR. {parts[0]} {parts[1]}"

    if "AV." in text:
      parts = [part.strip() for part in text.split("AV.")]

      if len(parts) == 2:
        text= f"AV. {parts[0]} {parts[1]}"

    text=text.replace('GRAL.','GENERAL').replace('TTE.','TENIENTE').replace('CNEL.','CORONEL')
    text=text.strip()

    return text
  else:
    return float(np.nan)

#### 2.3.2.Transformacion de los datos

Se reviso las primeras las primeras 5  columnas **'ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD'** y no muestran errores como tal

In [83]:
df_lesiones_victimas.drop(columns=['Unnamed: 0'],inplace=True)

In [84]:
df_lesiones_victimas.columns=['ID hecho', 'AAA', 'MM', 'DD', 'FECHA', 'VEHICULO_VICTIMA', 'SEXO',
       'EDAD_VICTIMA', 'GRAVEDAD']

In [85]:
df_lesiones_victimas.head()

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD


In [86]:
#Cambiamos el formato de las fecha a tipo de dato datetime
df_lesiones_victimas['FECHA']=df_lesiones_victimas['FECHA'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())

In [87]:
#Seleccionamos las columnas que nos interesan por su gran volumen de 'SD'
columnas_modificar=['VEHICULO_VICTIMA','SEXO','EDAD_VICTIMA','GRAVEDAD']

In [88]:
#Reemplazaremos las etiquetas de SD con NaN
df_lesiones_victimas=df_lesiones_victimas.applymap(reemplazar_nan)

In [89]:
df_lesiones_victimas

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NaN,Varon,57,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,NaN,Varon,NaN,NaN
3,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Varon,45,NaN
4,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Mujer,45,NaN
...,...,...,...,...,...,...,...,...,...
27600,LC-2021-0451911,2021,9,11,2021-09-11,TRANSPORTE PUBLICO,Varon,87,NaN
27601,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,Mujer,60,NaN
27602,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,Mujer,32,NaN
27603,LC-2021-0201378,2021,5,2,2021-05-02,MOTO,Varon,32,NaN


In [90]:
#Transformamos la columna SEXO quitando los espacios y colocandolo en mayusculas
df_lesiones_victimas['SEXO']=df_lesiones_victimas['SEXO'].apply(lambda x: x.upper().strip() if isinstance(x,str) else x)

In [91]:
#Cambiamos a integer los datos de la columna EDAD
df_lesiones_victimas['EDAD_VICTIMA']=pd.to_numeric(df_lesiones_victimas['EDAD_VICTIMA'])

In [92]:
#Transformamos la columna GRAVEDAD quitando los espacios y colocandolo en mayusculas
df_lesiones_victimas['GRAVEDAD']=df_lesiones_victimas['GRAVEDAD'].apply(lambda x: x.upper().strip() if isinstance(x,str) else x)

In [93]:
df_lesiones_victimas

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NaN,VARON,57.0,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,NaN,VARON,NaN,NaN
3,LC-2019-0000082,2019,1,1,2019-01-01,NaN,VARON,45.0,NaN
4,LC-2019-0000082,2019,1,1,2019-01-01,NaN,MUJER,45.0,NaN
...,...,...,...,...,...,...,...,...,...
27600,LC-2021-0451911,2021,9,11,2021-09-11,TRANSPORTE PUBLICO,VARON,87.0,NaN
27601,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,MUJER,60.0,NaN
27602,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,MUJER,32.0,NaN
27603,LC-2021-0201378,2021,5,2,2021-05-02,MOTO,VARON,32.0,NaN


Observamos que existe una gran cantidad de SD definidos como no definir sin embargo para nuestro analisis lo consideraremos como NaN y posteriormente haremos un **dropna** para poder procesar los datos

## **3.Exportacion**

In [94]:
path="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Data_Clean/lesiones_victimas.parquet.gzip"

In [95]:
df_lesiones_victimas.to_parquet(path,compression="gzip")